In [10]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import time

from utils import MLP
from utils import CMA_info
import utils

import torch.nn as nn

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Custom Model

In [12]:
randin = torch.rand((1,3,32,32))

In [13]:
layer_1 = nn.Conv2d(3,64,5)
layer_2 = nn.MaxPool2d(2)
layer_7 = nn.AdaptiveAvgPool2d((3,3))
#         layer_3 = nn.Conv2d(64,64,3)
#         layer_4 = nn.MaxPool2d(2)
#         layer_5 = nn.Conv2d(64,64,3)

In [14]:
layer_7(layer_1(randin)).shape

torch.Size([1, 64, 3, 3])

In [15]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        layer_1 = nn.Conv2d(3,64,5)
        layer_2 = nn.MaxPool2d(2)
#         layer_3 = nn.Conv2d(64,64,3)
#         layer_4 = nn.MaxPool2d(2)
#         layer_5 = nn.Conv2d(64,64,3)
#         layer_6 = nn.MaxPool2d(2)
        layer_7 = nn.AdaptiveAvgPool2d((3,3))
        layer_8 = nn.Dropout(p=0.5,inplace=False)
        layer_9 = nn.Linear(64*3*3, 2)
#         layer_10 = nn.Linear(64,2)
        
#         layer_9 = nn.Linear(64*3*3, 512)
#         layer_10 = nn.Linear(512,128)
#         layer_11 = nn.Linear(128,2)

        self.features_list = [layer_1, nn.ReLU(inplace=True),layer_7, nn.Flatten()]
        self.classifier_list = [layer_9]
        

#         self.features_list = [layer_1, nn.ReLU(inplace=True),layer_2,
#                               layer_3, nn.ReLU(inplace=True),layer_4,
#                               layer_5, nn.ReLU(inplace=True),layer_6, 
#                               layer_7, nn.Flatten()]
#         self.classifier_list = [layer_8,layer_9, nn.ReLU(inplace=True),
#                                 layer_10, nn.ReLU(inplace=True),
#                                 layer_11, nn.ReLU(inplace=True)]
        
        self.features = nn.Sequential(*self.features_list)
        self.classifier = nn.Sequential(*self.classifier_list)
        
    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [16]:
custom_model = CustomModel()

# Train Model

In [17]:
def get_uniform_limits(mean,std):
    A = 3.45*std #b-a
    B = mean*2 #a+b
    b = (B+A)/2
    a = b - A
    return a,b

input_shape = ((50,3,32,32))

test_min = -1
test_max= 1

test_limits = [test_min, test_max]

num_samples = 50

In [18]:
limits_1 = get_uniform_limits(0.2,0.225)
limits_2 = get_uniform_limits(0.7,0.225)

limits = limits_1 + limits_2

In [19]:
limits = [-10,10,20,40]

In [20]:
BATCH_SIZE = 100

In [33]:
custom_model = CustomModel()

In [34]:
custom_model.to(device)

CustomModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): AdaptiveAvgPool2d(output_size=(3, 3))
    (3): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Linear(in_features=576, out_features=2, bias=True)
  )
)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(custom_model.parameters(), lr=0.0001)

In [36]:
dset = utils.make_image_samples([5000,3,32,32], limits)

In [37]:
s = np.arange(dset[0].shape[0])
np.random.shuffle(s)

In [38]:
for epoch in tqdm(range(100)):  # loop over the dataset multiple times
    corrects = 0
    totals = 0
    for iter_ in range(10):
        noise_dset = [dset[0][s][iter_*1000:iter_*1000+1000], dset[1][s][iter_*1000:iter_*1000+1000]]
        noisy_inputs = noise_dset[0].to(device)
        noisy_labels = noise_dset[1].long().to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = custom_model(noisy_inputs)

        preds = torch.argmax(output,dim=1)
        corrects += torch.sum(noisy_labels == preds)
        totals += len(noisy_labels)

        loss = criterion(output, noisy_labels)
        loss.backward()
        optimizer.step()
    if epoch % 5 ==0:
        print(loss.item(), corrects/totals)
    if corrects/totals>0.99 and epoch > 10:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

0.19238229095935822 tensor(1., device='cuda:0')
0.09828417748212814 tensor(1., device='cuda:0')
0.06392990797758102 tensor(1., device='cuda:0')


In [39]:
corrects/totals

tensor(1., device='cuda:0')

# CMA Attack

In [40]:
# loaded_model = torch.load('alexnet_only_noise.pt')

In [41]:
model_to_attack = custom_model

In [43]:
attack_output = utils.cma_experiment_3d(model_to_attack, (1,3,32,32), test_limits, limits, 5, sigma_0 = 0.01,verb_disp=False)

  0%|          | 0/5 [00:00<?, ?it/s]